In [7]:
from bs4 import BeautifulSoup
import urllib.request,sys,time
import requests
import pandas as pd
import re
import math
import os

### Scraper for The Treasury Cloud Advisory

##### Uses URL of searched query, loops through each page, takes the title and link, and scans link to get the date of publication

### Check if document can get date. If cannot, then just skip it.

In [51]:
# Getting total results
url = 'https://search.treasury.gov/search?affiliate=treas&page=1&query=cloud+advisory'
try:
     # this might throw an exception if something goes wrong.
     page = requests.get(url) 
 # this describes what to do if an exception is thrown 
except Exception as e:    

    # get the exception information
    error_type, error_obj, error_info = sys.exc_info()      

    #print the link that cause the problem
    print ('ERROR FOR LINK:',url)

    #print error info and line that threw the exception                          
    print (error_type, 'Line:', error_info.tb_lineno)
        
soup = BeautifulSoup(page.text, "html.parser")
titles = soup.find_all('h4',attrs={'class':'title'})
results_count = soup.find_all('li',attrs={'id':'results-count'})
for a in results_count:
    results_count = int(a.text.strip().split(' ')[0])
print("Total results: ", results_count)

if results_count % len(titles) != 0:
    total_pages = math.ceil(results_count / len(titles))
else:
    total_pages = math.floor(results_count / len(titles))
    
print("Total number of pages: ", total_pages)

titles_list = list()
links_list = list()
dates_list = list()


for page in range(1, total_pages+1):
    
    # Base URL
    url = 'https://search.treasury.gov/search?affiliate=treas&page={pagenum}&query=cloud+advisory'.format(pagenum=page)
    try:
         # this might throw an exception if something goes wrong.
         page = requests.get(url) 
     # this describes what to do if an exception is thrown 
    except Exception as e:    

        # get the exception information
        error_type, error_obj, error_info = sys.exc_info()      

        #print the link that cause the problem
        print ('ERROR FOR LINK:',url)

        #print error info and line that threw the exception                          
        print (error_type, 'Line:', error_info.tb_lineno)
        continue

    soup = BeautifulSoup(page.text, "html.parser")

    # Getting all links and titles
    titles = soup.find_all('h4',attrs={'class':'title'})
    links=soup.find_all('span',attrs={'class':'url'})

    # Adding titles and links to list
    for a in titles:
        titles_list.append([b.strip() for b in a.text.strip().split('\n') if b.strip()])

    for a in links:
        links_list.append([b.strip() for b in a.text.strip().split('\n') if b.strip()])


# Create list for dates and add to dataframe
for link in links_list:
    url = link[0]
    print(url)
    if url.__contains__('pdf'):
        dates_list.append('Find yourself')
        continue
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.text, "html.parser")
    except Exception as e:
        error_type, error_obj, error_info = sys.exc_info()
        print ('ERROR FOR LINK:',url)                       
        print (error_type, 'Line:', error_info.tb_lineno)
        dates_list.append('Find yourself')
        continue

    date = soup.find('div', class_='date-format field field--name-field-news-publication-date field--type-datetime field--label-hidden field__item')
    if date is None:
        dates_list.append('Find yourself')
        continue
    else:
        dates_list.append(date.text.strip())
    print(date)
    
# Create dataframe for links and titles
d = {'Title': titles_list, 'Link': links_list, 'Date':dates_list}
print(len(titles_list))
print(len(links_list))
print(len(dates_list))

df = pd.DataFrame(data = d)

# Importing to csv
df.to_csv(r'C:\Users\65936\\OneDrive\Desktop\MAS\\USDeptOfTheTreasury.csv')

Total results:  146
Total number of pages:  8
https://home.treasury.gov/news/press-releases/jy1252
<div class="date-format field field--name-field-news-publication-date field--type-datetime field--label-hidden field__item"><time datetime="2023-02-08T12:00:00Z">February 8, 2023</time>
</div>
https://home.treasury.gov/news/press-releases/jl0248
<div class="date-format field field--name-field-news-publication-date field--type-datetime field--label-hidden field__item"><time datetime="2015-11-04T12:00:00Z">November 4, 2015</time>
</div>
https://home.treasury.gov/services/government-shared-services/enterprise-business-solutions/enterprise-content-management-ecm
https://home.treasury.gov/news/press-releases/jy1298
<div class="date-format field field--name-field-news-publication-date field--type-datetime field--label-hidden field__item"><time datetime="2023-02-24T22:03:46Z">February 24, 2023</time>
</div>
https://home.treasury.gov/system/files/266/FY-2021-CJ.pdf
https://home.treasury.gov/sites

<div class="date-format field field--name-field-news-publication-date field--type-datetime field--label-hidden field__item"><time datetime="2016-02-01T12:00:00Z">February 1, 2016</time>
</div>
https://home.treasury.gov/system/files/311/report.pdf
https://home.treasury.gov/system/files/236/FY2013-EO_13636_Assessment_Report.pdf
https://home.treasury.gov/system/files/266/FY-2016-BiB-complete.pdf
https://home.treasury.gov/system/files/241/ODCP-Annual-Report-for-Fiscal-Year-2017.pdf
https://home.treasury.gov/system/files/266/08.-Cybersecurity-Enhancement-Account-FY-2017-CJ.pdf
https://home.treasury.gov/system/files/266/25.-BEP-FY-2017-CJ.pdf
https://home.treasury.gov/system/files/266/11.-SIGTARP-FY-2017-CJ.pdf
https://home.treasury.gov/system/files/266/13.-CDFI-FY-2017-CJ.pdf
https://home.treasury.gov/news/press-releases/jl2596
<div class="date-format field field--name-field-news-publication-date field--type-datetime field--label-hidden field__item"><time datetime="2014-08-04T12:00:00Z">Aug